# Análise de Gênero nos Cursos de Exatas e Humanas - Censo de 2022

Este projeto realiza uma análise das matrículas e conclusões de cursos de Exatas e Humanas a partir dos microdados do Censo da Educação Superior 2022, com foco em questões de gênero.
O objetivo é compreender como homens e mulheres estão distribuídos em diferentes áreas do conhecimento, usando técnicas de visualização e testes estatísticos.


## 1. Importação de Bibliotecas

Aqui, importamos as bibliotecas essenciais para a análise de dados, como Pandas para manipulação de DataFrames, NumPy para operações numéricas, e Matplotlib para visualização.


In [ ]:
import pandas as pd
import numpy as np 
import scipy.stats as stats
from statsmodels.stats.proportion import proportions_ztest

## 2. Carregamento dos Dados

Vamos carregar os microdados do Censo da Educação Superior 2022 para iniciar nossa análise. Esses dados incluem informações sobre matrículas e conclusões de cursos em diversas áreas de estudo.


In [ ]:

df = pd.read_csv(r'C:\Users\Gabriel\Desktop\Projeto - Professores\data\data_raw\microdados_censo_da_educacao_superior_2022\microdados_educação_superior_2022\dados\MICRODADOS_CADASTRO_CURSOS_2022.CSV', 
                 sep=';', encoding='latin1', low_memory=False)

## 3. Tratamento dos Dados

Aqui, realizamos de acordo com os objetivos do projeto, o tratamento e ordenação dos dados, definindo variaveis e etc.


In [ ]:
# Verificando a primeira linha dos dados

print(df.head())

# Imprimindo todas colunas

df.columns.tolist()

# Agrupar por curso e gênero

cursos_populares = df.groupby('NO_CURSO').agg({
    'QT_MAT_FEM': 'sum',  # Total de matrículas femininas
    'QT_MAT_MASC': 'sum',  # Total de matrículas masculinas
}).reset_index()

# Ordenar para ver os cursos mais populares

cursos_populares = cursos_populares.sort_values(by='QT_MAT_FEM', ascending=False)

# Selecionar os top 10 cursos mais populares

top_cursos = cursos_populares.head(10)

In [ ]:
# Verifique os valores únicos na coluna NO_CINE_AREA_GERAL

print(df['NO_CINE_AREA_GERAL'].unique())

In [ ]:
# Filtrar por exatas e humanas
exatas = ['Ciências naturais, matemática e estatística', 'Engenharia, produção e construção', 'Computação e Tecnologias da Informação e Comunicação (TIC)']
humanas = ['Negócios, administração e direito', 'Educação', 'Ciências sociais, comunicação e informação', 'Artes e humanidades']

# Filtrar os cursos de exatas e humanas
df_exatas = df[df['NO_CINE_AREA_GERAL'].isin(exatas)]
df_humanas = df[df['NO_CINE_AREA_GERAL'].isin(humanas)]

# Top 10 cursos de exatas com mais matrículas
top_exatas = df_exatas.groupby('NO_CURSO').agg({
    'QT_MAT_FEM': 'sum',
    'QT_MAT_MASC': 'sum'
}).reset_index()
top_exatas['TOTAL_MATRICULAS'] = top_exatas['QT_MAT_FEM'] + top_exatas['QT_MAT_MASC']
top_exatas = top_exatas.sort_values(by='TOTAL_MATRICULAS', ascending=False).head(10)

# Top 10 cursos de humanas com mais matrículas
top_humanas = (
    df_humanas[df_humanas['NO_CURSO'].isin(['Educação Física', 'Matemática']) == False]
    .groupby('NO_CURSO')
    .agg({
        'QT_MAT_FEM': 'sum', 
        'QT_MAT_MASC': 'sum',
    })
    .assign(TOTAL_MATRICULAS=lambda x: x['QT_MAT_FEM'] + x['QT_MAT_MASC'])
    .sort_values(by='TOTAL_MATRICULAS', ascending=False)
    .head(10)
)

## 4. Algumas Estatísticas

Faremos alguns cálculos estatísticos que podem nos fornecer informações interessantes.


In [ ]:
# Exatas
total_exatas = df_exatas['QT_MAT_FEM'].sum() + df_exatas['QT_MAT_MASC'].sum() 

proporcao_fem_exatas = df_exatas['QT_MAT_FEM'].sum() / total_exatas

proporcao_masc_exatas = df_exatas['QT_MAT_MASC'].sum() / total_exatas

# Humanas
total_humanas = df_humanas['QT_MAT_FEM'].sum() + df_humanas['QT_MAT_MASC'].sum()
proporcao_fem_humanas = df_humanas['QT_MAT_FEM'].sum() / total_humanas
proporcao_masc_humanas = df_humanas['QT_MAT_MASC'].sum() / total_humanas

print(f"Proporção de Mulheres em Exatas: {proporcao_fem_exatas:.2f}")
print(f"Proporção de Homens em Exatas: {proporcao_masc_exatas:.2f}")
print(f"Proporção de Mulheres em Humanas: {proporcao_fem_humanas:.2f}")
print(f"Proporção de Homens em Humanas: {proporcao_masc_humanas:.2f}")

print(f"Número total de Exatas: {total_exatas}")
print(f"Número total de Humanas: {total_humanas}")


Proporção de Mulheres em Exatas: 0.27
Proporção de Homens em Exatas: 0.73
Proporção de Mulheres em Humanas: 0.60
Proporção de Homens em Humanas: 0.40
Número total de Exatas: 520598
Número total de Humanas: 1307685


In [ ]:
# Função para calcular intervalo de confiança de uma proporção
def intervalo_confianca_proporcao(p, n, z=1.96):
    erro_padrao = np.sqrt((p * (1 - p)) / n)
    intervalo_inferior = p - z * erro_padrao
    intervalo_superior = p + z * erro_padrao
    return intervalo_inferior, intervalo_superior

def formatar_intervalo(intervalo):
    return f'({intervalo[0]:.4f}, {intervalo[1]:.4f})'

# Intervalos de confiança
ic_mulheres_exatas = intervalo_confianca_proporcao(proporcao_fem_exatas, total_exatas)
ic_homens_exatas = intervalo_confianca_proporcao(proporcao_masc_exatas, total_exatas)
ic_mulheres_humanas = intervalo_confianca_proporcao(proporcao_fem_humanas, total_humanas)
ic_homens_humanas = intervalo_confianca_proporcao(proporcao_masc_humanas, total_humanas)

# Exibir os resultados
print("Intervalo de Confiança para Mulheres em Exatas:", formatar_intervalo(ic_mulheres_exatas))
print("Intervalo de Confiança para Homens em Exatas:", formatar_intervalo(ic_homens_exatas))
print("Intervalo de Confiança para Mulheres em Humanas:", formatar_intervalo(ic_mulheres_humanas))
print("Intervalo de Confiança para Homens em Humanas:", formatar_intervalo(ic_homens_humanas))


Intervalo de Confiança para Mulheres em Exatas: (0.2662, 0.2686)
Intervalo de Confiança para Homens em Exatas: (0.7314, 0.7338)
Intervalo de Confiança para Mulheres em Humanas: (0.5980, 0.5996)
Intervalo de Confiança para Homens em Humanas: (0.4004, 0.4020)


In [ ]:

homens_exatas = total_exatas * proporcao_masc_exatas  # Número de homens em Exatas
homens_humanas = total_humanas * proporcao_masc_humanas  # Número de homens em Humanas


# Contadores e observações
count = np.array([homens_exatas, homens_humanas])
nobs = np.array([total_exatas, total_humanas])

# Teste de hipóteses
stat, p_value = proportions_ztest(count, nobs)

print(f"Estatística do teste: {stat:.2f}")
print(f"Valor-p: {p_value:.5f}")

if p_value < 0.05:
    print("As proporções de homens em Exatas e Humanas são significativamente diferentes.")
else:
    print("Não há diferença significativa nas proporções de homens entre Exatas e Humanas.")


Estatística do teste: 404.46
Valor-p: 0.00000
As proporções de homens em Exatas e Humanas são significativamente diferentes.


In [ ]:
# Número de mulheres em Exatas e Humanas
mulheres_exatas = total_exatas * proporcao_fem_exatas
mulheres_humanas = total_humanas * proporcao_fem_humanas

# Contadores e observações
count = np.array([mulheres_exatas, mulheres_humanas])
nobs = np.array([total_exatas, total_humanas])

# Teste de hipóteses
stat, p_value = proportions_ztest(count, nobs)

print(f"Estatística do teste: {stat:.2f}")
print(f"Valor-p: {p_value:.5f}")

if p_value < 0.05:
    print("As proporções de mulheres em Exatas e Humanas são significativamente diferentes.")
else:
    print("Não há diferença significativa nas proporções de mulheres entre Exatas e Humanas.")

Estatística do teste: -404.46
Valor-p: 0.00000
As proporções de mulheres em Exatas e Humanas são significativamente diferentes.


## 5. Análise Exploratória e Visualizações

Aqui, realizamos a análise exploratória e visualizações dos dados, observando as matrículas e conclusões em cursos, com uma ênfase especial na divisão por gênero.


In [ ]:
# Top 10 Cursos por Número de Matrículas e Gênero

plt.figure(figsize=(14, 8))
plt.barh(top_cursos['NO_CURSO'], top_cursos['QT_MAT_FEM'], color='lightcoral', label='Matrículas Femininas')
plt.barh(top_cursos['NO_CURSO'], top_cursos['QT_MAT_MASC'], color='lightblue', left=top_cursos['QT_MAT_FEM'], label='Matrículas Masculinas')

plt.xlabel('Número de Matrículas')
plt.ylabel('Curso')
plt.title('Top 10 Cursos por Número de Matrículas e Gênero')
plt.legend()
plt.grid(axis='x', linestyle='--', alpha=0.7)
plt.show()


In [ ]:
# Top 10 Cursos de Humanas com Mais Matrículas por Gênero 

plt.figure(figsize=(14, 8))
plt.barh(top_humanas.index, top_humanas['QT_MAT_FEM'], color='lightcoral', label='Matrículas Femininas')
plt.barh(top_humanas.index, top_humanas['QT_MAT_MASC'], color='lightblue', left=top_humanas['QT_MAT_FEM'], label='Matrículas Masculinas')

# Títulos e rótulos
plt.xlabel('Número de Matrículas')
plt.ylabel('Curso')
plt.title('Top 10 Cursos de Humanas com Mais Matrículas por Gênero')
plt.legend()

# Grid e visualização
plt.grid(axis='x', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

In [ ]:
# Top 10 Cursos de Exatas com Mais Matrículas por Gênero

plt.figure(figsize=(14, 8))
plt.barh(top_exatas['NO_CURSO'], top_exatas['QT_MAT_FEM'], color='lightcoral', label='Matrículas Femininas')
plt.barh(top_exatas['NO_CURSO'], top_exatas['QT_MAT_MASC'], color='lightblue', left=top_exatas['QT_MAT_FEM'], label='Matrículas Masculinas')

# Títulos e rótulos
plt.xlabel('Número de Matrículas')
plt.ylabel('Curso')
plt.title('Top 10 Cursos de Exatas com Mais Matrículas por Gênero')
plt.legend()

# Grid e visualização
plt.grid(axis='x', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

## 6. Conclusão

Após a análise dos dados de matrículas e conclusões nos cursos de Exatas e Humanas, observamos algumas tendências importantes em relação à distribuição de gênero. Este estudo fornece uma base para entender as dinâmicas de gênero nas escolhas educacionais no Brasil.
